<hr>
<hr>

<div style="text-align: center;">
    <h1>SISTEMA DE RECOMENDAÇÃO DE FILME</h1>
    <p>>> Machine Learning <<</p>
</div>

<hr>
<hr>

Dados >> https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

In [74]:
# importando bibliotecas
import pandas as pd
import numpy as np

In [75]:
# importar arquivo com filmes e visualizar primeiras linhas
filmes = pd.read_csv('movies_metadata.csv', low_memory=False )
filmes.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [76]:
# Importando arquivo de avaliações e avaliando as primeiras linhas
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head(3)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523


## Pré-Processamento de Dados

In [77]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis

# Seleciona somente as variaveis que iremos utilizar
filmes = filmes[['id','original_title', 'original_language', 'vote_count']]

# Renomeia as variaveis
filmes.rename(columns= {
    'id' : 'ID_FILME',
    'original_title' : 'TITULO',
    'original_language' : 'LINGUAGEM',
    'vote_count' : 'QT_AVALIACOES'
}, inplace=True)

# Exibe as primeiras Linhas do arquivo tratado
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


In [78]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis (AVALIAÇÕES)

# Seleciona somente as variaveis que iremos utilizar
avaliacoes = avaliacoes[['userId', 'movieId', 'rating']]

# Renomeia as variaveis
avaliacoes.rename(columns= {
    'userId' : 'ID_USUARIO',
    'movieId' : 'ID_FILME',
    'rating' : 'AVALIAÇÃO'
}, inplace=True)

# Exibe as primeiras Linhas do arquivo tratado
avaliacoes.head()

,ID_USUARIO,ID_FILME,AVALIAÇÃO
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [79]:
# Tratando valores nulos
filmes.isna().sum()

ID_FILME          0
TITULO            0
LINGUAGEM        11
QT_AVALIACOES     6
dtype: int64

In [80]:
# Como são poucos valores nulos, vamos dropar eles porque não terá impacto
filmes.dropna(inplace=True)
filmes.isna().sum()

ID_FILME         0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [81]:
# Verificando se há valores nulos nas avaliaçãoes
avaliacoes.isna().sum()

ID_USUARIO    0
ID_FILME      0
AVALIAÇÃO     0
dtype: int64

In [82]:
# Quantidade de avaliações por usuario 
avaliacoes['ID_USUARIO'].value_counts()

ID_USUARIO
45811     18276
8659       9279
270123     7638
179792     7515
228291     7410
          ...  
30155         1
9641          1
164717        1
243426        1
234625        1
Name: count, Length: 270896, dtype: int64

In [83]:
# Vamos usar o ID_USUARIO que fizeram mais de 999 avaliações (Veremos os usuarios que avaliaram varios filmes, pois esses são fieis nas avaliações)
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

(2509,)

## Pegando somente avaliações dos usuarios que avaliaram mais de 999 vezes

In [84]:
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]
avaliacoes

,ID_USUARIO,ID_FILME,AVALIAÇÃO
17291,229,1,3.0
17292,229,2,3.0
17293,229,4,2.0
17294,229,5,1.0
17295,229,7,2.0
...,...,...,...
26023517,270887,171439,1.0
26023518,270887,171755,3.0
26023519,270887,173149,5.0
26023520,270887,173405,5.0


## Pegando somente filmes que foram avaliados mais de 999 vezes

In [85]:
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

filmes.shape

(1121, 4)

## Agrupar e visualizar a quantidade de filmes pela linguagem

In [86]:
linguagem = filmes['LINGUAGEM'].value_counts()
linguagem

LINGUAGEM
en    1100
fr       5
ja       5
it       3
ko       2
pt       1
de       1
es       1
cn       1
sv       1
id       1
Name: count, dtype: int64

## Selecionar apenas os filmes em EN (inglish)

In [87]:
filmes = filmes[filmes['LINGUAGEM'] == 'en']
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
5,949,Heat,en,1886.0
9,710,GoldenEye,en,1194.0
15,524,Casino,en,1343.0


Pegando informações desse DF

In [88]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   object 
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB


## ``Precisamos arrumar o "ID_FILME"`` pois ele deve ser int

In [89]:
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   int32  
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), int32(1), object(2)
memory usage: 38.7+ KB


### Para juntar os dois DF (Filmes + Avaliações) Precisamos que sejam do mesmo ```tipo (Dtype)``

In [90]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3844582 entries, 17291 to 26023521
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID_USUARIO  int64  
 1   ID_FILME    int64  
 2   AVALIAÇÃO   float64
dtypes: float64(1), int64(2)
memory usage: 117.3 MB


In [91]:
avaliacoes['ID_FILME'] = avaliacoes['ID_FILME'].astype(int)
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3844582 entries, 17291 to 26023521
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID_USUARIO  int64  
 1   ID_FILME    int32  
 2   AVALIAÇÃO   float64
dtypes: float64(1), int32(1), int64(1)
memory usage: 102.7 MB


<hr>

## Now we'll merge the dataframes

<hr>

In [92]:
avaliacoes_e_filmes = avaliacoes.merge(filmes, on = "ID_FILME")

avaliacoes_e_filmes.tail()

,ID_USUARIO,ID_FILME,AVALIAÇÃO,TITULO,LINGUAGEM,QT_AVALIACOES
189877,243443,8909,1.0,Wanted,en,2583.0
189878,218368,101299,4.0,The Hunger Games: Catching Fire,en,6656.0
189879,219677,146216,1.0,RED 2,en,1548.0
189880,221327,118340,4.0,Guardians of the Galaxy,en,10014.0
189881,248600,157336,1.5,Interstellar,en,11187.0


In [93]:
# verificando  a quantidade de filmes pelo tamanho do arquivo
avaliacoes_e_filmes.shape

"""
    O DF FICOU MAIOR PORQUE UM MESMO USUARIO PODE AVALIAR VARIOS FILMES
"""

'\n    O DF FICOU MAIOR PORQUE UM MESMO USUARIO PODE AVALIAR VARIOS FILMES\n'

In [94]:
# Verificando se ha valores nulos
avaliacoes_e_filmes.isna().sum()

ID_USUARIO       0
ID_FILME         0
AVALIAÇÃO        0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

## Removendo valores duplicados do Dataframe (DF)

Pois um mesmo usuario pode ter avaliado o mesmo filme varias vezes

In [95]:
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO','ID_FILME'], inplace=True)

# verificando tamanho novamente
avaliacoes_e_filmes.shape

(189882, 6)

## Remover colunas que não serão utilizadas

In [96]:
del avaliacoes_e_filmes['ID_FILME']

In [97]:
avaliacoes_e_filmes.tail(15)

,ID_USUARIO,AVALIAÇÃO,TITULO,LINGUAGEM,QT_AVALIACOES
189867,243546,2.0,Enemy at the Gates,en,1023.0
189868,101273,2.5,A.I. Artificial Intelligence,en,2011.0
189869,139811,0.5,A.I. Artificial Intelligence,en,2011.0
189870,175551,4.0,A.I. Artificial Intelligence,en,2011.0
189871,210002,3.0,A.I. Artificial Intelligence,en,2011.0
189872,139811,0.5,Brokeback Mountain,en,1531.0
189873,175551,4.0,Brokeback Mountain,en,1531.0
189874,210002,3.0,Brokeback Mountain,en,1531.0
189875,163016,2.0,Teenage Mutant Ninja Turtles,en,2677.0
189876,167844,2.5,Wanted,en,2583.0


<hr>

## Fazer o PIVOT
<hr>

In [98]:
# Preciso que cada usuario seja uma variavel (coluna), para conseguir atribuir cada nota por usuario
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns="ID_USUARIO", index="TITULO", values="AVALIAÇÃO")

# Visualizando pivot
filmes_pivot.head(15)

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.5,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
127 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN
2 Fast 2 Furious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey,NaN,NaN,3.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN
27 Dresses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28 Days Later,3.0,NaN,2.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,2.5,3.0,NaN,NaN,3.0,NaN
28 Weeks Later,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,...,NaN,0.5,3.0,1.0,2.0,3.0,NaN,2.5,NaN,NaN


## Remover valores nulos do pivot

In [99]:
filmes_pivot.fillna(0, inplace=True)
filmes_pivot.head()

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.5,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
127 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.5,2.0,0.0,0.0,0.0,0.0
2 Fast 2 Furious,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<hr>

## Aplicando ``Machine Learning``
<hr>

In [100]:
# Importando a biblioteca csr_matrix do pacote Scipy
# Isso possibilita criar uma matriz sparsa -> -> ->
from scipy.sparse import csr_matrix

"""
Matriz sparsa é uma matriz aonde esse algoritmo irá perceber todos os campos que tem '0' e marcar esses valores e 'excluir' entre aspas esses valores para que fique um aprendizado mais fluido. Faz como uma 'compressao desses valores' """

# Transformando dataset em matriz sparsa
filmes_sparse = csr_matrix(filmes_pivot)

# Verificando type
type(filmes_sparse)

scipy.sparse._csr.csr_matrix

In [101]:
# Importando o algoritmo KNN do SciKit Learn
from sklearn.neighbors import NearestNeighbors

# Criando e treinando modelo preditivo
modelo = NearestNeighbors(algorithm= 'brute') # um dos algoritmos da biblioteca
modelo.fit(filmes_sparse)

# *SERÁ CRIADO EM SEGUNDOS PORQUE JÁ FIZEMOS O SPARSE DELE*

NearestNeighbors(algorithm='brute')

 <div style="text-align: center;">
    <h1>Fazendo previsões de sugestão de filmes</h1>
</div>

In [102]:
# 127 Hours
distancia, sugestoes = modelo.kneighbors(filmes_pivot.filter(items=['127 Hours'], axis=0).values.reshape(1, -1))

for i in range(len(sugestoes)):
    display(filmes_pivot.index[sugestoes[i]])

Index(['127 Hours', 'American Hustle', 'The Expendables 2', 'Lord of War',
       'RED 2'],
      dtype='object', name='TITULO')

In [103]:
filmes_pivot.index[:100]

Index(['10 Things I Hate About You', '12 Angry Men', '127 Hours', '1408',
       '2 Fast 2 Furious', '2001: A Space Odyssey', '27 Dresses',
       '28 Days Later', '28 Weeks Later', '300', '3:10 to Yuma',
       '4: Rise of the Silver Surfer', '50 First Dates', '8 Mile',
       'A Beautiful Mind', 'A Clockwork Orange', 'A Nightmare on Elm Street',
       'A.I. Artificial Intelligence', 'AVP: Alien vs. Predator',
       'Ace Ventura: Pet Detective', 'Airplane!', 'Aladdin', 'Alien', 'Aliens',
       'Alvin and the Chipmunks', 'Amadeus', 'American Beauty',
       'American Gangster', 'American History X', 'American Hustle',
       'American Pie', 'American Pie 2', 'American Psycho', 'American Wedding',
       'Anchorman: The Legend of Ron Burgundy', 'Annie Hall', 'Ant-Man',
       'Antz', 'Apocalypse Now', 'Apocalypto', 'Apollo 13', 'Armageddon',
       'Atonement', 'Austin Powers: International Man of Mystery',
       'Avengers: Age of Ultron', 'Babel', 'Back to the Future',
       'Back

In [ ]:
# Escolha o filme e adicione no campo em branco "items=['']"
distancia, sugestoes = modelo.kneighbors(filmes_pivot.filter(items=[''], axis=0).values.reshape(1,-1))

for i in range(len(sugestoes)):
    display(filmes_pivot.index[sugestoes[i]])